<a href="https://colab.research.google.com/github/wojtekgradzinski/WojtekRepo/blob/main/fashion_NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!pip install -U spacy 3.1

In [9]:
!python -m spacy download en_core_web_lg

     |████████████████████████████████| 777.1 MB 13 kB/s 
  Attempting uninstall: en-core-web-lg
    Found existing installation: en-core-web-lg 2.2.5
    Uninstalling en-core-web-lg-2.2.5:
      Successfully uninstalled en-core-web-lg-2.2.5
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
spacy.__version__

'3.1.3'

In [12]:
# Import libraries
import spacy
import random
from spacy.util import minibatch, compounding
from pathlib import Path
from spacy.training import Example

nlp = spacy.load("en_core_web_lg")

In [14]:
#2nd model


with open("/content/drive/MyDrive/Colab Notebooks/7. Natural Language Processing/6. NER/fashion brands.txt") as file:
    dataset = file.read()

doc = nlp(dataset)
print("Entities:", [(ent.text, ent.label_) for ent in doc.ents])

Entities: [('Jodie Turner-Smith', 'PERSON'), ('Gucci', 'PERSON'), ('Loewe', 'ORG'), ('Jodie Turner-Smith', 'PERSON'), ('Chanel', 'ORG'), ('Atelier Versace', 'ORG'), ('Diane Kruger', 'PERSON'), ('Alberta', 'GPE'), ('Prada', 'ORG'), ('Jean Paul Gaultier', 'PERSON'), ('Jodie Foster', 'PERSON'), ('H&M', 'ORG'), ('Diodato', 'PERSON'), ('Venetian', 'NORP'), ('Giorgio Armani', 'PERSON'), ('The Golden Lion', 'ORG'), ('Serena Rossi', 'PERSON'), ('Emporio Armani', 'ORG'), ('two', 'CARDINAL'), ('Alba Parietti', 'PERSON'), ('Gessica Notaro', 'PERSON'), ("Wonder Woman's", 'ORG'), ('Paradise Island', 'LOC'), ('Donatella Finocchiaro', 'PERSON'), ('Miu Miu', 'ORG'), ('Giorgio Armani', 'PERSON'), ('Preis', 'ORG'), ('Chanel', 'ORG'), ('Venetian', 'NORP'), ('Janelle Monae', 'PERSON'), ('Cynthia Erivo', 'PERSON'), ('the 91st Academy Awards', 'ORG'), ('Billy Porter', 'PERSON'), ('the 91st Academy Awards', 'ORG'), ('Saoirse Ronan', 'PERSON'), ('Margot Robbie', 'PERSON'), ('Chanel', 'ORG'), ('Emma Stone', 'P

In [15]:
#2nd model


with open("/content/drive/MyDrive/Colab Notebooks/7. Natural Language Processing/6. NER/fashion brands.txt") as file:
    dataset = file.read()

doc = nlp(dataset)
print("Entities:", [(ent.text, ent.label_) for ent in doc.ents])

"""

STEP 1 - TRAIN DATA

"""

# Prepare training data

words = ["Gucci", "Chanel", "Schiaparelli", "Balenciaga",
         "Loewe", "Saint Laurent", "basil", "Atelier Versace",
         "Alberta Ferretti", "Lanvin", "Ferretti", "Prada", "Jean Paul Gaultier",
         "Givenchy", "Louis Vuitton", "Louis Vuitton", "Burberry", "H&M",
         "Armani Privé", "Emporio Armani", "Emporio Armani", "Emporio Armani",
         "Emporio Armani", "Jayp", "Alexander McQueen", "Calvin Klein"]

train_data = []

with open("/content/drive/MyDrive/Colab Notebooks/7. Natural Language Processing/6. NER/fashion brands.txt") as file:
    dataset = file.readlines()
    for sentence in dataset:
        print("######")
        print("sentence: ", sentence)
        print("######")
        sentence = sentence.lower()
        entities = []
        for word in words:
            word = word.lower()
            if word in sentence:
                start_index = sentence.index(word)
                end_index = len(word) + start_index
                print("word: ", word)
                print("----------------")
                print("start index:", start_index)
                print("end index:", end_index)
                pos = (start_index, end_index,'ORG')
                entities.append(pos)
        element = (sentence.rstrip('\n'), {"entities": entities})

        train_data.append(element)
        print('----------------')
        print("element:", element)

        # ("this is my sentence", {"entities": [0, 4, "PREP"]})
        # ("this is my sentence", {"entities": [6, 8, "VERB"]})

"""

STEP 2 - UPDATE MODEL

"""

ner = nlp.get_pipe("ner")

for _, annotations in train_data:
    for ent in annotations.get("entities"):
        ner.add_label(ent[2])

# Train model

pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
unaffected_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]

with nlp.disable_pipes(*unaffected_pipes):
    for iteration in range(30):
        print("Iteration #", iteration)

        random.shuffle(train_data)
        losses = {}

        batches = minibatch(train_data, size=compounding(4.0, 32.0, 1.001))
        for batch in batches:
            for text, annotations in batch:
                doc = nlp.make_doc(text)
                example = Example.from_dict(doc, annotations)
                nlp.update([example], losses=losses, drop=0.1)
        print("Losses:", losses)



# Save the model

output_dir = Path("/ner/")
nlp.to_disk(output_dir)
print("Saved correctly!")

"""

STEP 3 - TEST THE UPDATED MODEL

"""

print("Loading model...")
nlp_updated = spacy.load(output_dir)

# old sentence, old word
doc = nlp_updated("Cynthia Erivo attends the 92nd Academy Awards, Designer: Versace, Year: 2020")
print("entities:", [(ent.text, ent.label_) for ent in doc.ents])

# old sentence, new word
doc = nlp_updated("Lady Gaga attends the 91st Academy Awards, Designer: Ted Baker, Year: 2019")
print("entities:", [(ent.text, ent.label_) for ent in doc.ents])

# new sentence, new word
doc = nlp_updated("I am a big fan of Tommy Hilfigher")
print("entities:", [(ent.text, ent.label_) for ent in doc.ents])

# new sentence, no word
doc = nlp_updated("Wojtek loves trading crypto!")
print("entities:", [(ent.text, ent.label_) for ent in doc.ents])

Entities: [('Jodie Turner-Smith', 'PERSON'), ('Gucci', 'PERSON'), ('Loewe', 'ORG'), ('Jodie Turner-Smith', 'PERSON'), ('Chanel', 'ORG'), ('Atelier Versace', 'ORG'), ('Diane Kruger', 'PERSON'), ('Alberta', 'GPE'), ('Prada', 'ORG'), ('Jean Paul Gaultier', 'PERSON'), ('Jodie Foster', 'PERSON'), ('H&M', 'ORG'), ('Diodato', 'PERSON'), ('Venetian', 'NORP'), ('Giorgio Armani', 'PERSON'), ('The Golden Lion', 'ORG'), ('Serena Rossi', 'PERSON'), ('Emporio Armani', 'ORG'), ('two', 'CARDINAL'), ('Alba Parietti', 'PERSON'), ('Gessica Notaro', 'PERSON'), ("Wonder Woman's", 'ORG'), ('Paradise Island', 'LOC'), ('Donatella Finocchiaro', 'PERSON'), ('Miu Miu', 'ORG'), ('Giorgio Armani', 'PERSON'), ('Preis', 'ORG'), ('Chanel', 'ORG'), ('Venetian', 'NORP'), ('Janelle Monae', 'PERSON'), ('Cynthia Erivo', 'PERSON'), ('the 91st Academy Awards', 'ORG'), ('Billy Porter', 'PERSON'), ('the 91st Academy Awards', 'ORG'), ('Saoirse Ronan', 'PERSON'), ('Margot Robbie', 'PERSON'), ('Chanel', 'ORG'), ('Emma Stone', 'P

ValueError: ignored